In [1]:
import os
data_path = os.getcwd()
print(f"{data_path=}")

data_path='/app/notebooks/rapprochements/Marseille pôle rénovation urbaine'


In [2]:
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def list_columns(filepath):
    df = gpd.read_file(filepath)
    print(f"Processing file: {filepath}")
    print(df.columns)

def process_gpkg_files(directory):
    dfs = []
    for filename in os.listdir(directory):
        if filename.endswith('.gpkg'):
            filepath = os.path.join(directory, filename)
            df = gpd.read_file(filepath)
            if 'id_rnb' in df.columns:
                dfs.append(df)
            else:
                print(f"No id_rnb in {filename}")
    
    return merge_gdf(dfs)

def merge_gdf(gdf_list):
    if not gdf_list:
        raise ValueError("The list of GeoDataFrames is empty.")

    # Find common columns across all GeoDataFrames
    common_columns = set(gdf_list[0].columns)
    for gdf in gdf_list[1:]:
        common_columns.intersection_update(gdf.columns)

    # Print columns that will be dropped for each GeoDataFrame
    for i, gdf in enumerate(gdf_list):
        dropped_columns = set(gdf.columns) - common_columns
        for col in dropped_columns:
            print(f"Dropping column '{col}' from GeoDataFrame {i+1}")

    # Keep only common columns in each GeoDataFrame
    merged_gdfs = [gdf[list(common_columns)] for gdf in gdf_list]

    # Merge all GeoDataFrames
    merged = gpd.GeoDataFrame(pd.concat(merged_gdfs, ignore_index=True))

    return merged

# Example usage:
directory = data_path  # Replace with your directory path
merged_dfs = process_gpkg_files(directory)
print(merged_dfs.columns)
check_duplicate_objectids(merged_dfs)

No id_rnb in amest_amest_pdv_perimetres_projet_npnru_amp.gpkg
No id_rnb in amest_amest_pdv_projets_24_amenagement_amp.gpkg
Dropping column 'situ_qp' from GeoDataFrame 1
Dropping column 'nlogcree' from GeoDataFrame 1
Dropping column 'dest' from GeoDataFrame 1
Dropping column 'nom_ope' from GeoDataFrame 1
Dropping column 'nb_acc_soc' from GeoDataFrame 1
Dropping column 'sup_ter' from GeoDataFrame 1
Dropping column 'etage' from GeoDataFrame 1
Dropping column 'contr_al' from GeoDataFrame 1
Dropping column 'nb_loc_lib' from GeoDataFrame 1
Dropping column 'moa' from GeoDataFrame 1
Dropping column 'propri_ini' from GeoDataFrame 1
Dropping column 'type_autre' from GeoDataFrame 1
Dropping column 'nb_acc_lib' from GeoDataFrame 1
Dropping column 'financmt' from GeoDataFrame 1
Dropping column 'nb_lls' from GeoDataFrame 1
Dropping column 'sdp_estim' from GeoDataFrame 1
Dropping column 'nom_qp' from GeoDataFrame 1
Dropping column 'nb_autre' from GeoDataFrame 1
Dropping column 'nb_residsp' from GeoDa

In [4]:
df = gpd.read_file(os.path.join(data_path, 'amest_pdv_projets_residentialisation_amp.gpkg'))

print("nombre de bâtiments in amest_pdv_projets_residentialisation_amp_df : ", len(df))
print(df.columns)

nombre de bâtiments in amest_pdv_projets_residentialisation_amp_df :  381
Index(['OBJECTID', 'secteur', 'nom_bat', 'avancmt', 'debut_trav', 'fin_trav',
       'num_conv', 'id_top', 'id_sop', 'nom_qpv', 'code_qp', 'id_uniq',
       'id_bdt', 'id_rnb', 'ct_amp', 'commune', 'codecomm', 'codeinsee',
       'nom_donnee', 'datemaj', 'source', 'codesiret', 'shape_Length',
       'shape_Area', 'geometry'],
      dtype='object')


In [13]:
# print current crs
print("current crs : ", df.crs)

# convert from 2154 to 4326
df = df.to_crs(epsg=4326)

current crs :  EPSG:4326


In [6]:
filtered_df = df.drop_duplicates(subset = 'id_uniq')

In [7]:
ids = filtered_df["id_uniq"]
print(ids[ids.duplicated()])

Series([], Name: id_uniq, dtype: object)


In [8]:
from shapely import to_geojson
import json

def row_to_input(df_row_raw):
    df_row = dict(df_row_raw)
        
    return {
        "ext_id" : df_row["id_uniq"],
        "polygon": json.loads(to_geojson(df_row["geometry"]))
    }

inputs = list(filtered_df.apply(row_to_input, axis=1))

In [9]:
import random
sample_inputs = random.sample(inputs, 10)
print(sample_inputs[0])

{'ext_id': 'BATIMENT0000000258060910', 'polygon': {'type': 'MultiPolygon', 'coordinates': [[[[5.432289760627534, 43.51897372581665], [5.432266087233705, 43.51891124686145], [5.4322587107308, 43.51891231240294], [5.432234923166622, 43.518847135586014], [5.43224229966151, 43.518846070046195], [5.432217466763741, 43.518785417366765], [5.431967237115003, 43.51883513466989], [5.431993419751501, 43.51889845756054], [5.4320007581846825, 43.518896492751104], [5.432027055106041, 43.51896251349458], [5.432018480900748, 43.51896450601238], [5.43203509952753, 43.51900643997792], [5.431907645915303, 43.51903450140694], [5.432069668450848, 43.51944313288039], [5.4321960774975615, 43.51941959539278], [5.432043351739588, 43.51902605803112], [5.432289760627534, 43.51897372581665]]]]}}


In [11]:
from batid.services.guess_bdg_new import Guesser, PartialRoofHandler
guesses_path = "./guesses.json"
guesser = Guesser(batch_size=50)
guesser.handlers = [
    PartialRoofHandler(),
]
guesser.create_work_file(list(inputs), guesses_path)
guesser.guess_work_file(guesses_path)

  0%|          | 0/8 [00:00<?, ?it/s]

Partial roof with multiple polygons is not supported
Partial roof with multiple polygons is not supported
Partial roof with multiple polygons is not supported
Partial roof with multiple polygons is not supported
Partial roof with multiple polygons is not supported


In [12]:
guesser = Guesser()
guesser.load_work_file(guesses_path)
guesser.report()

-- Report --
Number of rows: 378
Number of match: 367 (97.09%)

-- finished_steps --
Rows with finished_steps closest_from_point: 0 (0.00%)
Rows with finished_steps geocode_address: 0 (0.00%)
Rows with finished_steps geocode_name: 0 (0.00%)
Rows with empty finished_steps: 0 (0.00%)

-- match_reasons : absolute --
match_reason
sole_bdg_intersects_roof_enough     323
many_bdgs_covered_enough_by_roof     44
Name: count, dtype: int64

-- match_reasons : % --
match_reason
sole_bdg_intersects_roof_enough     85.449735
many_bdgs_covered_enough_by_roof    11.640212
Name: count, dtype: float64

-- Inputs --
